# Imports

In [171]:
import numpy as np
import pandas as pd 

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.datasets import make_regression

import re
from datetime import datetime

# Loading Data

In [172]:
train_path = "data/train.csv"
test_path = "data/test_x.csv"

train_raw = pd.read_csv(train_path)
test_raw = pd.read_csv(test_path)

train_raw = train_raw.dropna(subset=['Degerlendirme Puani'])

train_y_raw = train_raw['Degerlendirme Puani']
train_X_raw = train_raw.drop('Degerlendirme Puani', axis=1)

test_X_raw = test_raw

/tmp/ipykernel_52644/753989495.py:4: DtypeWarning: Columns (13,15,19,30,32,37,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  train_raw = pd.read_csv(train_path)


# Preprocessing

## Features

In [173]:
dropped = ['Lise Adi', 'Lise Adi Diger', 'Lise Sehir', 'Lise Turu', 'Lise Bolumu', 'Lise Bolum Diger', 'Lise Mezuniyet Notu', 
           'Burs Aldigi Baska Kurum', 'Baska Kurumdan Aldigi Burs Miktari', 'Anne Egitim Durumu', 'Baba Egitim Durumu', 'Uye Oldugunuz Kulubun Ismi', 'Aktif olarak bir STK üyesi misiniz?',
           "Hangi STK'nin Uyesisiniz?", 'Stk Projesine Katildiniz Mi?', 'Girisimcilikle Ilgili Deneyiminiz Var Mi?',
           'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?', 'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite', 'id']
train_X_raw = train_X_raw.drop(dropped, axis=1)
test_X_raw = test_X_raw.drop(dropped, axis=1)

In [174]:
tr_chars = {'İ': 'I', 'Ü': 'U', 'Ö': 'O', 'Ğ': 'G', 'Ş': 'S', 'Ç': 'C'}

### Doğum tarihi ve Yaş

In [175]:
def tarih_formatla(df):
    def extract_year(date_str):
        if not isinstance(date_str, str):
            return None
        
        # Farklı tarih formatları için regex pattern'leri
        patterns = [
            r'(\d{4})[-/.](\d{1,2})[-/.](\d{1,2})',  # YYYY-MM-DD, YYYY/MM/DD, YYYY.MM.DD
            r'(\d{1,2})[-/.](\d{1,2})[-/.](\d{4})',  # DD-MM-YYYY, DD/MM/YYYY, DD.MM.YYYY
            r'(\d{2})[-/.](\d{1,2})[-/.](\d{1,2})',  # YY-MM-DD, YY/MM/DD, YY.MM.DD
            r'(\d{1,2})[-/.](\d{1,2})[-/.](\d{2})'   # DD-MM-YY, DD/MM/YY, DD.MM.YY
        ]
        
        for pattern in patterns:
            match = re.search(pattern, date_str)
            if match:
                groups = match.groups()
                if len(groups[0]) == 4:  # YYYY MM DD formatı
                    year = int(groups[0])
                elif len(groups[2]) == 4:  # DD MM YYYY formatı
                    year = int(groups[2])
                else:  # YY-MM-DD veya DD-MM-YY formatı
                    year = int(groups[0] if len(groups[0]) == 2 else groups[2])
                    year += 1900 if year > 24 else 2000
                return year
        
        return np.nan
    
    def yas_sinirla(entry):
        dogum_tarihi = entry['Dogum Tarihi']
        basvurudugu_yil = entry['Basvuru Yili']
        return dogum_tarihi if 1900 <= dogum_tarihi <= basvurudugu_yil - 18 else np.nan
    
    # Yılları çıkar
    df['Dogum Tarihi'] = df["Dogum Tarihi"].apply(extract_year)
    df['Dogum Tarihi'] = df['Dogum Tarihi'].apply(lambda x: x if 1900 <= x <= 2007 else np.nan)
    df['yas'] = 2024 - df['Dogum Tarihi']
    # Geçersiz yılları ortalama ile doldur
    mean_year = df['yas'].mean()
    df['yas'] = df['yas'].fillna(round(mean_year)).astype(int)
    
    
    return df

train_X_raw['Dogum Tarihi'] = train_X_raw['Dogum Tarihi'].replace('1/1/70 2:00', np.nan)
# Başvuru yılı ve Doğum tarihi feature ı kullanarak yaş feature ı üretildi.
train_X_raw = tarih_formatla(train_X_raw)
test_X_raw = tarih_formatla(test_X_raw)

train_X_raw.drop("Dogum Tarihi", axis=1, inplace=True)
test_X_raw.drop("Dogum Tarihi", axis=1, inplace=True)
#test_X_raw["Dogum Tarihi"].value_counts(dropna=False).to_csv("tsetf.csv")
#train_X_raw["yas"].value_counts()

### Cinsiyet(One Hot)

In [176]:
train_X_raw['Cinsiyet'] = train_X_raw['Cinsiyet'].str.lower()
train_X_raw['Cinsiyet'] = train_X_raw['Cinsiyet'].replace(to_replace=[np.nan, 'belirtmek istemiyorum'], value='diger')
test_X_raw['Cinsiyet'] = test_X_raw['Cinsiyet'].str.lower()
test_X_raw['Cinsiyet'] = test_X_raw['Cinsiyet'].replace(to_replace=[np.nan, 'belirtmek istemiyorum'], value='diger')

#train_X_raw["Cinsiyet"].value_counts(dropna=False).to_csv("train.csv")
#test_X_raw["Cinsiyet"].value_counts(dropna=False).to_csv("test.csv")

### Doğum Yeri(Frequency) ve İkametgah Şehri(One Hot)

In [177]:
from rapidfuzz import process

sehirler=["Adana", "Adıyaman", "Afyon", "Ağrı", "Amasya", "Ankara", "Antalya", "Artvin", "Aydın", "Balıkesir", "Bilecik", "Bingöl", "Bitlis", "Bolu", "Burdur", "Bursa", "Çanakkale", "Çankırı", "Çorum", "Denizli", "Diyarbakır", "Edirne", "Elazığ", "Erzincan", "Erzurum", "Eskişehir", "Gaziantep", "Giresun", "Gümüşhane", "Hakkari", "Hatay", "Isparta", "İçel (Mersin)", "İstanbul", "İzmir", "Kars", "Kastamonu", "Kayseri", "Kırklareli", "Kırşehir", "Kocaeli", "Konya", "Kütahya", "Malatya", "Manisa", "Kahramanmaraş", "Mardin", "Muğla", "Muş", "Nevşehir", "Niğde", "Ordu", "Rize", "Sakarya", "Samsun", "Siirt", "Sinop", "Sivas", "Tekirdağ", "Tokat", "Trabzon", "Tunceli", "Şanlıurfa", "Uşak", "Van", "Yozgat", "Zonguldak", "Aksaray", "Bayburt", "Karaman", "Kırıkkale", "Batman", "Şırnak", "Bartın", "Ardahan", "Iğdır", "Yalova", "Karabük", "Kilis", "Osmaniye", "Düzce"
"KKTC"]

def temizle_metin(metin):
    if pd.isna(metin):
        return metin
    metin = str(metin).lower().strip()
    metin = re.sub(r'[^\w\s]', '', metin)
    return metin
# EŞİK DEĞİŞTİRİLEBİLİR
def en_yakin_eslesme(metin, secenekler, esik=70):
    if pd.isna(metin):
        return np.nan
    en_iyi_eslesme = process.extractOne(metin, secenekler)
    if en_iyi_eslesme[1] >= esik:
        return en_iyi_eslesme[0]
    return np.nan

def sehir_adi_temizle_ve_standartlastir(df, column_name):
    df[column_name] = df[column_name].apply(temizle_metin)
    df[column_name] = df[column_name].apply(lambda x: en_yakin_eslesme(x, sehirler))
    
    return df

train_X_raw = sehir_adi_temizle_ve_standartlastir(train_X_raw, 'Dogum Yeri')
train_X_raw['Dogum Yeri'] = train_X_raw['Dogum Yeri'].fillna('diger')
test_X_raw = sehir_adi_temizle_ve_standartlastir(test_X_raw, 'Dogum Yeri')
test_X_raw['Dogum Yeri'] = test_X_raw['Dogum Yeri'].fillna('diger')


train_X_raw = sehir_adi_temizle_ve_standartlastir(train_X_raw, 'Ikametgah Sehri')
train_X_raw['Ikametgah Sehri'] = train_X_raw['Ikametgah Sehri'].fillna('diger')
test_X_raw = sehir_adi_temizle_ve_standartlastir(test_X_raw, 'Ikametgah Sehri')
test_X_raw['Ikametgah Sehri'] = test_X_raw['Ikametgah Sehri'].fillna('diger')

frequency = train_X_raw['Dogum Yeri'].value_counts()
train_X_raw["encoded_dogum_yeri"] = train_X_raw['Dogum Yeri'].map(frequency)
frequency = test_X_raw['Dogum Yeri'].value_counts()
test_X_raw["encoded_dogum_yeri"] = test_X_raw['Dogum Yeri'].map(frequency)

train_X_raw.drop('Dogum Yeri', axis=1, inplace=True)
test_X_raw.drop('Dogum Yeri', axis=1, inplace=True)

#train_X_raw['Ikametgah Sehri'].value_counts(dropna=False)

### Üniversite Adı(Frequency)

In [178]:
train_X_raw['Universite Adi'] = train_X_raw['Universite Adi'].fillna('DIGER')
train_X_raw['Universite Adi'] = train_X_raw['Universite Adi'].str.upper()
for tr_char, en_char in tr_chars.items():
    train_X_raw['Universite Adi'] = train_X_raw['Universite Adi'].apply(lambda x: str(x).replace(tr_char, en_char))

test_X_raw['Universite Adi'] = test_X_raw['Universite Adi'].fillna('DIGER')
test_X_raw['Universite Adi'] = test_X_raw['Universite Adi'].str.upper()
for tr_char, en_char in tr_chars.items():
    test_X_raw['Universite Adi'] = test_X_raw['Universite Adi'].apply(lambda x: str(x).replace(tr_char, en_char))


frequency = train_X_raw['Universite Adi'].value_counts()
train_X_raw["encoded_universite_adi"] = train_X_raw['Universite Adi'].map(frequency)

frequency = test_X_raw['Universite Adi'].value_counts()
test_X_raw["encoded_universite_adi"] = test_X_raw['Universite Adi'].map(frequency)

train_X_raw.drop('Universite Adi', axis=1, inplace=True)
test_X_raw.drop('Universite Adi', axis=1, inplace=True)

#train_X_raw['Universite Adi'].value_counts(dropna=False).to_csv("a.csv")
#train_X_raw["encoded_universite_adi"].value_counts(dropna=False)


### Üniversite Türü(One Hot)

In [179]:
train_X_raw["Universite Turu"] = train_X_raw["Universite Turu"].str.upper()
train_X_raw["Universite Turu"] = train_X_raw["Universite Turu"].fillna('DIGER')

test_X_raw["Universite Turu"] = test_X_raw["Universite Turu"].str.upper()
test_X_raw["Universite Turu"] = test_X_raw["Universite Turu"].fillna('DIGER')

#train_X_raw["Universite Turu"].value_counts(dropna=False).to_csv("train.csv")
#test_X_raw["Universite Turu"].value_counts(dropna=False).to_csv("test.csv")

### 'Burslu ise Burs Yuzdesi'(Binning), 'Burs Aliyor mu?(One Hot)

In [180]:
train_X_raw["Burs Aliyor mu?"] = train_X_raw["Burs Aliyor mu?"].str.upper()
train_X_raw["Burs Aliyor mu?"] = train_X_raw["Burs Aliyor mu?"].fillna("HAYIR")
test_X_raw["Burs Aliyor mu?"] = test_X_raw["Burs Aliyor mu?"].str.upper()
test_X_raw["Burs Aliyor mu?"] = test_X_raw["Burs Aliyor mu?"].fillna("HAYIR")

train_X_raw.loc[train_X_raw['Burs Aliyor mu?'] == 'HAYIR', 'Burslu ise Burs Yuzdesi'] = int(0)
test_X_raw.loc[test_X_raw['Burs Aliyor mu?'] == 'HAYIR', 'Burslu ise Burs Yuzdesi'] = int(0)
train_X_raw['Burslu ise Burs Yuzdesi'] = train_X_raw['Burslu ise Burs Yuzdesi'].fillna(int(0))
test_X_raw['Burslu ise Burs Yuzdesi'] = test_X_raw['Burslu ise Burs Yuzdesi'].fillna(int(0))

cut_labels = ['25altı', '25-49', '50-74', '75-99', '100']
cut_bins = [-1, 24.0, 49.0, 74.0, 99.0, 100.0]

train_X_raw['binned_burs_yuzde'] = pd.cut(train_X_raw['Burslu ise Burs Yuzdesi'], bins=cut_bins, labels=cut_labels)
test_X_raw['binned_burs_yuzde'] = pd.cut(test_X_raw['Burslu ise Burs Yuzdesi'], bins=cut_bins, labels=cut_labels)
train_X_raw.drop('Burslu ise Burs Yuzdesi', axis=1, inplace=True)
test_X_raw.drop('Burslu ise Burs Yuzdesi', axis=1, inplace=True)

#train_X_raw['binned_burs_yuzde'].value_counts(dropna=False)

### Bölüm(Frequnecy?)

In [181]:
train_X_raw['Bölüm'] = train_X_raw['Bölüm'].str.upper()
test_X_raw['Bölüm'] = test_X_raw['Bölüm'].str.upper()
for tr_char, en_char in tr_chars.items():
    train_X_raw['Bölüm'] = train_X_raw['Bölüm'].apply(lambda x: str(x).replace(tr_char, en_char))
for tr_char, en_char in tr_chars.items():
    test_X_raw['Bölüm'] = test_X_raw['Bölüm'].apply(lambda x: str(x).replace(tr_char, en_char))

train_X_raw['Bölüm'] = train_X_raw['Bölüm'].fillna('DIGER')
train_X_raw['Bölüm'] = train_X_raw['Bölüm'].replace(r'^\s*$', 'DIGER', regex=True)
train_X_raw['Bölüm'] = train_X_raw['Bölüm'].replace('nan', 'DIGER')

test_X_raw['Bölüm'] = test_X_raw['Bölüm'].fillna('DIGER')
test_X_raw['Bölüm'] = test_X_raw['Bölüm'].replace(r'^\s*$', 'DIGER', regex=True)
test_X_raw['Bölüm'] = test_X_raw['Bölüm'].replace('nan', 'DIGER')

#value_counts = train_X_raw['Bölüm'].value_counts()
#top_10 = value_counts.nlargest(10)

#train_X_raw['Bölüm'] = train_X_raw['Bölüm'].apply(lambdsa x: x if x in top_10.index else 'DIGER')
frequency = train_X_raw['Bölüm'].value_counts()
train_X_raw['encoded_bolum'] = train_X_raw['Bölüm'].map(frequency)
frequency = test_X_raw['Bölüm'].value_counts()
test_X_raw['encoded_bolum'] = test_X_raw['Bölüm'].map(frequency)

train_X_raw.drop('Bölüm', axis=1, inplace=True)
test_X_raw.drop('Bölüm', axis=1, inplace=True)

### Universite Kaçıncı Sınıf

In [182]:
train_X_raw['Universite Kacinci Sinif'] = train_X_raw['Universite Kacinci Sinif'].str.upper()
for tr_char, en_char in tr_chars.items():
    train_X_raw['Universite Kacinci Sinif'] = train_X_raw['Universite Kacinci Sinif'].apply(lambda x: str(x).replace(tr_char, en_char))
test_X_raw['Universite Kacinci Sinif'] = test_X_raw['Universite Kacinci Sinif'].str.upper()
for tr_char, en_char in tr_chars.items():
    test_X_raw['Universite Kacinci Sinif'] = test_X_raw['Universite Kacinci Sinif'].apply(lambda x: str(x).replace(tr_char, en_char)) 

train_X_raw['Universite Kacinci Sinif'] = train_X_raw['Universite Kacinci Sinif'].apply(lambda x: str(x).replace("0", "HAZIRLIK"))
train_X_raw['Universite Kacinci Sinif'] = train_X_raw['Universite Kacinci Sinif'].apply(lambda x: str(x).replace("nan", "BILINMIYOR"))
test_X_raw['Universite Kacinci Sinif'] = test_X_raw['Universite Kacinci Sinif'].apply(lambda x: str(x).replace("0", "HAZIRLIK"))
test_X_raw['Universite Kacinci Sinif'] = test_X_raw['Universite Kacinci Sinif'].apply(lambda x: str(x).replace("nan", "BILINMIYOR"))
    

### Universite Not Ortalamasi

In [183]:
replacements = {
    '4.0-3.5'    : '3.50 - 4.00',
    '4-3.5'      : '3.50 - 4.00',
    '3.50-3,6179': '3.50 - 4.00',
    '3.5 - 4.0'  : '3.50 - 4.00',

    '3.50-3'     : '3.00 - 3.50',
    '3.00 - 3.49': '3.00 - 3.50',

    '3.00-2.50'  : '2.50 - 3.00',
    '2.50 - 2.99': '2.50 - 3.00',
    '2.50 -3.00' : '2.50 - 3.00',

    '1.80 - 2.49': '2.50 VE ALTI',
    '2.00 - 2.50': '2.50 VE ALTI',
    '1.00 - 2.50': '2.50 VE ALTI',
    '0 - 1.79'   : '2.50 VE ALTI', 

    'ORTALAMA BULUNMUYOR': 'YOK',
    'HAZIRLIGIM': 'YOK',
    'NOT ORTALAMASI YOK': 'YOK',
    'nan': 'YOK'
}

replacements_test = {
    '2.50 - 2.99': '2.50 - 3.00',
    '3.00 - 3.49': '3.00 - 3.50',
    '1.80 - 2.49': '2.50 VE ALTI',
    '0 - 1.79'   : '2.50 VE ALTI',

    'HAZIRLIGIM' : 'YOK'
}

train_X_raw['Universite Not Ortalamasi'] = train_X_raw['Universite Not Ortalamasi'].str.upper()
for tr_char, en_char in tr_chars.items():
    train_X_raw['Universite Not Ortalamasi'] = train_X_raw['Universite Not Ortalamasi'].apply(lambda x: str(x).replace(tr_char, en_char))
for key, value in replacements.items():
    train_X_raw['Universite Not Ortalamasi'] = train_X_raw['Universite Not Ortalamasi'].apply(lambda x: str(x).replace(key, value))

test_X_raw['Universite Not Ortalamasi'] = test_X_raw['Universite Not Ortalamasi'].str.upper()
for tr_char, en_char in tr_chars.items():
    test_X_raw['Universite Not Ortalamasi'] = test_X_raw['Universite Not Ortalamasi'].apply(lambda x: str(x).replace(tr_char, en_char))
for key, value in replacements_test.items():
    test_X_raw['Universite Not Ortalamasi'] = test_X_raw['Universite Not Ortalamasi'].apply(lambda x: str(x).replace(key, value))

value_counts = train_X_raw['Universite Not Ortalamasi'].value_counts()
top_5 = value_counts.nlargest(5)
train_X_raw['Universite Not Ortalamasi'] = train_X_raw['Universite Not Ortalamasi'].apply(lambda x: x if x in top_5.index else 'DIGER')

#train_X_raw['Universite Not Ortalamasi'].value_counts(dropna=False).to_csv("train.csv")
#test_X_raw['Universite Not Ortalamasi'].value_counts(dropna=False).to_csv("test.csv")

### Daha Once Baska Bir Universiteden Mezun Olmus

In [184]:
train_X_raw["Daha Once Baska Bir Universiteden Mezun Olmus"] = train_X_raw["Daha Once Baska Bir Universiteden Mezun Olmus"].replace(to_replace=np.nan, value="Hayır")
test_X_raw["Daha Once Baska Bir Universiteden Mezun Olmus"] = test_X_raw["Daha Once Baska Bir Universiteden Mezun Olmus"].replace(to_replace=np.nan, value="Hayır")

### 'Baska Bir Kurumdan Burs Aliyor mu?'

In [185]:
train_X_raw['Baska Bir Kurumdan Burs Aliyor mu?'] = train_X_raw['Baska Bir Kurumdan Burs Aliyor mu?'].fillna("Hayır")
test_X_raw['Baska Bir Kurumdan Burs Aliyor mu?'] = test_X_raw['Baska Bir Kurumdan Burs Aliyor mu?'].fillna("Hayır")
#train_X_raw['Baska Bir Kurumdan Burs Aliyor mu?'].value_counts(dropna=False)

#train_X_raw['Baska Kurumdan Aldigi Burs Miktari'] = train_X_raw['Baska Kurumdan Aldigi Burs Miktari'].str.replace(r'[^0-9]', '', regex=True)

#train_X_raw['Baska Kurumdan Aldigi Burs Miktari'].value_counts(dropna=False).to_csv("a.csv")


### Anne Calisma Durumu, Anne Sektor

In [186]:
#test_X_raw['Anne Sektor'].value_counts(dropna=False)
#train_X_raw['Anne Sektor'].value_counts(dropna=False)

In [187]:
train_X_raw['Anne Calisma Durumu'] = train_X_raw['Anne Calisma Durumu'].fillna("bilinmiyor")
test_X_raw['Anne Calisma Durumu'] = test_X_raw['Anne Calisma Durumu'].fillna("bilinmiyor")

train_X_raw['Anne Sektor'] = train_X_raw['Anne Sektor'].str.upper()
for tr_char, en_char in tr_chars.items():
    train_X_raw['Anne Sektor'] = train_X_raw['Anne Sektor'].apply(lambda x: str(x).replace(tr_char, en_char))
train_X_raw['Anne Sektor'] = train_X_raw['Anne Sektor'].apply(lambda x: x if x in ['OZEL SEKTOR', 'KAMU', 'DIGER'] else 'BELIRSIZ')

test_X_raw['Anne Sektor'] = test_X_raw['Anne Sektor'].str.upper()
for tr_char, en_char in tr_chars.items():
    test_X_raw['Anne Sektor'] = test_X_raw['Anne Sektor'].apply(lambda x: str(x).replace(tr_char, en_char))
test_X_raw['Anne Sektor'] = test_X_raw['Anne Sektor'].apply(lambda x: x if x in ['OZEL SEKTOR', 'KAMU', 'DIGER'] else 'BELIRSIZ')

#print(train_X_raw['Anne Sektor'].value_counts(dropna=False))

### Baba Calisma Durumu, Baba Sektor

In [188]:
train_X_raw['Baba Calisma Durumu'] = train_X_raw['Baba Calisma Durumu'].fillna("bilinmiyor")
test_X_raw['Baba Calisma Durumu'] = test_X_raw['Baba Calisma Durumu'].fillna("bilinmiyor")

train_X_raw['Baba Sektor'] = train_X_raw['Baba Sektor'].str.upper()
for tr_char, en_char in tr_chars.items():
    train_X_raw['Baba Sektor'] = train_X_raw['Baba Sektor'].apply(lambda x: str(x).replace(tr_char, en_char))

test_X_raw['Baba Sektor'] = test_X_raw['Baba Sektor'].str.upper()
for tr_char, en_char in tr_chars.items():
    test_X_raw['Baba Sektor'] = test_X_raw['Baba Sektor'].apply(lambda x: str(x).replace(tr_char, en_char))

train_X_raw['Baba Sektor'] = train_X_raw['Baba Sektor'].apply(lambda x: x if x in ['OZEL SEKTOR', 'KAMU', 'DIGER'] else 'BELIRSIZ')
test_X_raw['Baba Sektor'] = test_X_raw['Baba Sektor'].apply(lambda x: x if x in ['OZEL SEKTOR', 'KAMU', 'DIGER'] else 'BELIRSIZ')

### Kardeş Sayısı

In [189]:
train_X_raw['Kardes Sayisi'] = pd.to_numeric(train_X_raw['Kardes Sayisi'], errors='coerce').fillna(0).astype(int)
test_X_raw['Kardes Sayisi'] = pd.to_numeric(test_X_raw['Kardes Sayisi'], errors='coerce').fillna(0).astype(int)

### Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?

In [190]:
train_X_raw['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'] = train_X_raw['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'].fillna("Hayır")
test_X_raw['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'] = test_X_raw['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'].fillna("Hayır")

### Profesyonel Bir Spor Daliyla Mesgul musunuz?

In [191]:
train_X_raw['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] = train_X_raw['Profesyonel Bir Spor Daliyla Mesgul musunuz?'].fillna("Hayır")
test_X_raw['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] = test_X_raw['Profesyonel Bir Spor Daliyla Mesgul musunuz?'].fillna("Hayır")

### Spor Dalindaki Rolunuz Nedir?

In [192]:
replacements = {
    'KAPTAN / LİDER': 'KAPTAN',
    'Lider/Kaptan': 'KAPTAN',
    'Kaptan': 'KAPTAN',

    'Takım Oyuncusu': 'OYUNCU',
    'Bireysel': 'OYUNCU',

    'DİĞER': 'DIGER',
    'Diğer': 'DIGER',
}

replacements_test = {
    'Bireysel Spor' : 'OYUNCU',
    'Takım Oyuncusu': 'OYUNCU',

    'Lider/Kaptan'  : 'KAPTAN',
    'Diğer': 'DIGER'
}

for key, value in replacements.items():
    train_X_raw['Spor Dalindaki Rolunuz Nedir?'] = train_X_raw['Spor Dalindaki Rolunuz Nedir?'].apply(lambda x: str(x).replace(key, value))
for key, value in replacements_test.items():
    test_X_raw['Spor Dalindaki Rolunuz Nedir?'] = test_X_raw['Spor Dalindaki Rolunuz Nedir?'].apply(lambda x: str(x).replace(key, value))

train_X_raw.loc[train_X_raw['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] == 'Hayır', 'Spor Dalindaki Rolunuz Nedir?'] = 'nan'
test_X_raw.loc[test_X_raw['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] == 'Hayır', 'Spor Dalindaki Rolunuz Nedir?'] = 'nan'
train_X_raw['Spor Dalindaki Rolunuz Nedir?'] = train_X_raw['Spor Dalindaki Rolunuz Nedir?'].apply(lambda x: str(x).replace('nan', 'YOK'))
test_X_raw['Spor Dalindaki Rolunuz Nedir?'] = test_X_raw['Spor Dalindaki Rolunuz Nedir?'].apply(lambda x: str(x).replace('nan', 'YOK'))
#train_X_raw['Spor Dalindaki Rolunuz Nedir?'].value_counts(dropna=False)

### 'Ingilizce Biliyor musunuz?', 'Ingilizce Seviyeniz?'

In [193]:
train_X_raw['Ingilizce Biliyor musunuz?'] = train_X_raw['Ingilizce Biliyor musunuz?'].fillna("Hayır")
test_X_raw['Ingilizce Biliyor musunuz?'] = test_X_raw['Ingilizce Biliyor musunuz?'].fillna("Hayır")

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
train_X_raw['Ingilizce Seviyeniz?'] = imp.fit_transform(train_X_raw['Ingilizce Seviyeniz?'].values.reshape(-1,1))[:,0]
test_X_raw['Ingilizce Seviyeniz?'] = imp.transform(test_X_raw['Ingilizce Seviyeniz?'].values.reshape(-1,1))[:,0]
train_X_raw.loc[train_X_raw['Ingilizce Seviyeniz?'] != "0", 'Ingilizce Biliyor musunuz?'] = "Evet"
test_X_raw.loc[test_X_raw['Ingilizce Seviyeniz?'] != "0", 'Ingilizce Biliyor musunuz?'] = "Evet"
#train_X_raw['Ingilizce Seviyeniz?'].value_counts(dropna=False)

### DEVAM EDİCEM

## One Hot Encoding

In [194]:
encode_features = ['Ikametgah Sehri', 'Cinsiyet', 'Universite Turu', 'Burs Aliyor mu?', 'binned_burs_yuzde', 
                   'Universite Kacinci Sinif', 'Universite Not Ortalamasi', 'Daha Once Baska Bir Universiteden Mezun Olmus', 
                   'Baska Bir Kurumdan Burs Aliyor mu?', 'Anne Calisma Durumu', 'Baba Calisma Durumu', 'Baba Sektor' ,'Anne Sektor', 
                   'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?', 'Profesyonel Bir Spor Daliyla Mesgul musunuz?',
                   'Spor Dalindaki Rolunuz Nedir?', 'Ingilizce Biliyor musunuz?', 'Ingilizce Seviyeniz?']

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

encoded_train = encoder.fit_transform(train_X_raw[encode_features])
categories = encoder.get_feature_names_out(encode_features)
encoded_df_train = pd.DataFrame(encoded_train, columns=categories)

encoded_test = encoder.transform(test_X_raw[encode_features])
encoded_df_test = pd.DataFrame(encoded_test, columns=categories)

train_X_raw = pd.concat([train_X_raw, encoded_df_train], axis=1)
test_X_raw = pd.concat([test_X_raw, encoded_df_test], axis=1)

train_X_raw.drop(encode_features, axis=1, inplace=True)
test_X_raw.drop(encode_features, axis=1, inplace=True)

In [195]:
with open('columns_output.txt', 'w') as f:
    for col in train_X_raw.columns:
        f.write(f"{col}\n")

# Eğitim

In [196]:
train_X = train_X_raw
train_y = train_y_raw

test_X = test_X_raw

model = RandomForestRegressor(random_state=42)

#kfold = KFold(n_splits=5, shuffle=True, random_state=42)
#scores = cross_val_score(model, train_X, train_y, cv=kfold, scoring='neg_mean_squared_error')
#rmse_scores = np.sqrt(-scores)

X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)

val_pred = model.predict(X_val)
val_rmse = np.sqrt(((val_pred - y_val) ** 2).mean())
print("Test Seti RMSE:", val_rmse)

#print("5-Fold Çapraz Doğrulama RMSE Skorları:", rmse_scores)
#print("Ortalama RMSE:", rmse_scores.mean())

Test Seti RMSE: 9.292160018918755


In [198]:
predictions = model.predict(test_X)
output = pd.DataFrame({'id': test_raw['id'],
                       'Degerlendirme Puani': predictions})
output.to_csv('submission.csv', index=False)